In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
from simcore_sdk import node_ports
PORTS = node_ports.ports()

In [3]:
import pandas as pd
import matplotlib.pyplot as plt


def create_learned_model_input(path, plot_vs_tcnap):

    column_names = ['t_ms', 'CV', 'Vmax','M_mod', 'B_mod', 'tauSD']
    data = pd.read_csv(path, sep=',', names=column_names)



    dpi = 96
    height = 1024
    width = 1024
    fontsize = 16
    
    x_axis = data.CV
    if plot_vs_tcnap:
        x_axis = data.t_ms
        
        
    
    plt.figure(figsize=(width / dpi, height / dpi), dpi=dpi)    
    plt.subplot(4,1,1)
    plt.title(r'Learned Model Input Parameters', fontsize=20)
    plt.plot(x_axis, -1e12*data.Vmax, 'k.', markersize=5)
    plt.ylabel(r'$V_{MAX}$ $(\mu V)$', fontsize=fontsize)
    if plot_vs_tcnap:
        plt.xlim(0, max(data.t_ms))        
    else:
        plt.xlim(min(data.CV), 100)
        plt.xscale('log')
    plt.grid(which='both')

    plt.subplot(4,1,2)
    plt.plot(x_axis, data.M_mod, 'b.', markersize=5)
    plt.ylabel(r'M coeff $(\Delta_{Ist}/\Delta_\lambda$)', fontsize=fontsize)
    if plot_vs_tcnap:
        plt.xlim(0, max(data.t_ms))        
    else:
        plt.xlim(min(data.CV), 100)
        plt.xscale('log')
    plt.grid(which='both')

    plt.subplot(4,1,3)
    plt.plot(x_axis, data.B_mod, 'r.', markersize=5)
    plt.ylabel(r'B coeff $(mA)$', fontsize=fontsize)
    if plot_vs_tcnap:
        plt.xlim(0, max(data.t_ms))        
    else:
        plt.xlim(min(data.CV), 100)
        plt.xscale('log')
    plt.grid(which='both')

    plt.subplot(4,1,4)
    plt.plot(x_axis, data.tauSD, 'm.', markersize=5)
    plt.ylabel(r'$\tau_{SD}$ $(ms)$', fontsize=fontsize)
    plt.xlabel(r'Conduction Velocity $(m/s)$', fontsize=fontsize)
    if plot_vs_tcnap:
        plt.xlim(0, max(data.t_ms))        
    else:
        plt.xlim(min(data.CV), 100)
        plt.xscale('log')
    plt.grid(which='both')

    plt.show()


In [4]:
import ipywidgets as widgets

style = {"description_width": "initial"}

nerve_profile = widgets.Select(
    options=["Subject 1: Cervical Vagus", "Subject 2: Cervical Vagus", "Subject 2: Gastric Branch"],
    value="Subject 1: Cervical Vagus",
    # rows=10,
    description="Select:",
    disabled=False,
    style=style
)

load_input = widgets.Button(
    description="Load",
    disabled=False,
    button_style="success",
    align_self="right"
)

input_layout = widgets.VBox([nerve_profile, load_input])
input_label = widgets.Label("Select a Nerve Profile")
input_box = widgets.VBox([input_label, input_layout], layout=widgets.Layout(border="solid 1px"))

charge_phase_cb = widgets.Checkbox(
    value=False,
    description="Plot against Charge-Phase",
    disabled=False,
    style=style
)

time_cb = widgets.Checkbox(
    value=False,
    description="Plot CNAP versus Time (ms)",
    disabled=False,
    style=style
)

plot_options_layout = widgets.VBox([charge_phase_cb, time_cb])
plot_options_label = widgets.Label("Plot Options")
plot_options_box = widgets.VBox([plot_options_label, plot_options_layout], layout=widgets.Layout(border="solid 1px"))

horizontal_input_box = widgets.HBox([input_box, plot_options_box], layout=widgets.Layout(width="100%"))

out = widgets.Output(layout={'border': '1px solid black'})
vertical_box = widgets.VBox([horizontal_input_box, out], layout=widgets.Layout(height="100%"))


#@out.capture()
def on_load_clicked(b):
        out.clear_output()
        with out:
            print("Load clicked.", nerve_profile.value)
            model_id = nerve_profile.index + 1
            !execute_cnap.sh $model_id 0 0.0 1.0 0.5 0.4
            path = '/home/jovyan/outputs/input.csv'
            create_learned_model_input(path, plot_vs_tcnap=time_cb.value)
load_input.on_click(on_load_clicked)

vertical_box

In [5]:
import pandas as pd
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import numpy as np
%matplotlib notebook


def create_predicted_compound_nerve_action(cv_path, t_path, ist_path, tst_path, qst_path, vpred_path, lpred_path, fixed_tst, plot_vs_qst, plot_vs_tCNAP):
    data_cv = pd.read_csv(cv_path, sep=',', header=None)
    data_tcnap = pd.read_csv(t_path, sep=',', header=None)
    data_ist = None
    data_tst = None
    if fixed_tst:
        data_ist = pd.read_csv(ist_path, sep=',', header=None)
    else:
        data_tst = pd.read_csv(tst_path, sep=',', header=None)
    data_CAP = pd.read_csv(qst_path, sep=',', header=None)
    data_vpred = pd.read_csv(vpred_path, sep=',', header=None)
    data_lpred = pd.read_csv(lpred_path, sep=',', header=None)
    dpi = 96
    height = 1024
    width = 800
    fontsize = 16
    
    fig = plt.figure(figsize=(width / dpi, height / dpi), dpi=dpi)
    fig.suptitle(r'Predicted Compound Nerve Action Potentials')
    gs = fig.add_gridspec(2, 1)
    ax1 = fig.add_subplot(gs[0, 0], projection='3d')
    ax2 = fig.add_subplot(gs[1, 0])
    
    data_cv[data_cv>100] = None
    x_axis = data_cv
    x_label = r'CV (m/s)'
    x_scale = 'symlog'
    x_lim = [min(data_cv), 100]
    
    if plot_vs_tCNAP:
        x_axis = data_tcnap
        x_label = r'$t_{CNAP}$ (ms)'
        x_scale = 'linear'
        x_lim = [0, max(data_tcnap)]
    
    y_axis = data_ist
    y_label = r'$I_{st}$ (mA)'
    if not fixed_tst:
        y_axis = data_tst
        y_label = r'$t_{st}$ (mA)'
    if plot_vs_qst:
        y_axis = data_CAP
        y_label = r'$Q_{st}$ (nC)'
       

    ax1.set_xlabel(x_label)
    ax1.set_xscale(x_scale)
    ax1.set_xlim(x_lim)
    ax1.set_ylabel(y_label)
    ax1.set_zlabel(r'$V_{pred}$ $(\mu V)$')
    ax1.plot_wireframe(x_axis, y_axis, data_vpred, rstride=0, cstride=5)
    ax1.grid(which='both')

    ax2.set_xscale(x_scale)
    ax2.set_xlabel(x_label)
    ax2.set_ylabel(y_label)
    pcm = ax2.pcolormesh(np.nan_to_num(x_axis), y_axis, data_lpred)
    cbar = plt.colorbar(pcm, orientation='horizontal')
    cbar.ax.set_xlabel("%Activation")


In [6]:
from pathlib import Path
import shutil

def move_files_to_output():
    # move the file from /outputs/FILE.csv to /outputs/FILE/FILE.csv
    outputs_path = Path("~/outputs").expanduser()
    for path in outputs_path.glob("*.csv"):
        new_path = (path.parent / path.stem / path.name)
        new_path.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy2(src=path, dst=new_path)


In [7]:


start_ist = widgets.FloatText(
    value=0,
    description="Starting Ist (mA):",
    disabled=False,
    layout=widgets.Layout(width="200px"),
    style=style
)

end_ist = widgets.FloatText(
    value=1,
    description="Ending Ist (mA):",
    disabled=False,
    style=style
)

step_size_current = widgets.FloatText(
    value=0.01,
    description="Step Size (mA):",
    disabled=False,
    style=style
)

fixed_tst = widgets.FloatText(
    value=0.4,
    description="Fixed tst (ms):",
    disabled=False,
    style=style
)

predict_CNAPs_current = widgets.Button(
    description="Predict CNAPs",
    disabled=False,
    button_style="danger"
)

sweep_pulse_current_layout = widgets.VBox([start_ist, end_ist, step_size_current, fixed_tst, predict_CNAPs_current])
sweep_pulse_current_label = widgets.Label("Sweep Pulse Current")
sweep_pulse_current_box = widgets.VBox([sweep_pulse_current_label, sweep_pulse_current_layout], layout=widgets.Layout(border="solid 1px"))


start_tst = widgets.FloatText(
    value=0,
    description="Starting tst (mA):",
    disabled=False,
    style=style
)

end_tst = widgets.FloatText(
    value=1,
    description="Ending tst (mA):",
    disabled=False,
    style=style
)

step_size_duration = widgets.FloatText(
    value=0.01,
    description="Step Size (mA):",
    disabled=False,
    style=style
)

fixed_ist = widgets.FloatText(
    value=0.4,
    description="Fixed Ist (ms):",
    disabled=False,
    style=style
)

predict_CNAPs_time = widgets.Button(
    description="Predict CNAPs",
    disabled=False,
    button_style="info",
    layout=widgets.Layout(align="right")
)

sweep_pulse_duration_layout = widgets.VBox([start_tst, end_tst, step_size_duration, fixed_ist, predict_CNAPs_time])
sweep_pulse_duration_label = widgets.Label("Sweep Pulse Duration")
sweep_pulse_duration_box = widgets.VBox([sweep_pulse_duration_label, sweep_pulse_duration_layout], layout=widgets.Layout(border="solid 1px"))

parameters_layout = widgets.HBox([sweep_pulse_current_box, sweep_pulse_duration_box], layout=widgets.Layout(width="100%"))
out2 = widgets.Output(layout={'border': '1px solid black'})
vertical_box = widgets.VBox([parameters_layout, out2], layout=widgets.Layout(height="100%"))

def on_current_clicked(b):
    # Fixed tst
    out2.clear_output()
    model_id = nerve_profile.index + 1
    sweep_param = 1
    
    with out2:
        print("Current clicked.", charge_phase_cb.value, time_cb.value, start_ist.value, end_ist.value, step_size_current.value, fixed_tst.value)
        !execute_cnap.sh $model_id $sweep_param $start_ist.value $end_ist.value $step_size_current.value $fixed_tst.value        
        create_predicted_compound_nerve_action(cv_path='/home/jovyan/outputs/CV_plot.csv', t_path='/home/jovyan/outputs/t_plot.csv',  ist_path='/home/jovyan/outputs/Ist_plot.csv', tst_path='/home/jovyan/outputs/tst_plot.csv', qst_path='/home/jovyan/outputs/CAP_plot.csv', vpred_path='/home/jovyan/outputs/V_pred_plot.csv', lpred_path='/home/jovyan/outputs/Lpred_plot.csv', fixed_tst=True, plot_vs_qst=charge_phase_cb.value, plot_vs_tCNAP=time_cb.value), 
predict_CNAPs_current.on_click(on_current_clicked)


def on_time_clicked(b):
    # Fixed Ist
    out2.clear_output()
    model_id = nerve_profile.index + 1
    sweep_param = 0    
    with out2:
        print("Time clicked.", charge_phase_cb.value, time_cb.value, start_tst.value, end_tst.value, step_size_duration.value, fixed_ist.value)
        !execute_cnap.sh $model_id $sweep_param $start_ist.value $end_ist.value $step_size_current.value $fixed_tst.value        
        create_predicted_compound_nerve_action(cv_path='/home/jovyan/outputs/CV_plot.csv', t_path='/home/jovyan/outputs/t_plot.csv',  ist_path='/home/jovyan/outputs/Ist_plot.csv', tst_path='/home/jovyan/outputs/tst_plot.csv', qst_path='/home/jovyan/outputs/CAP_plot.csv', vpred_path='/home/jovyan/outputs/V_pred_plot.csv', lpred_path='/home/jovyan/outputs/Lpred_plot.csv', fixed_tst=False, plot_vs_qst=charge_phase_cb.value, plot_vs_tCNAP=time_cb.value), 
        
predict_CNAPs_time.on_click(on_time_clicked)



vertical_box